## Try-out notebook for playing around with vectorization

In [2]:
import numpy as np

#### How to stretch np arrays so row wise mult will be possible

In [3]:
x = np.array([1, 2])
x = np.tile(x[:, np.newaxis], 3).ravel()
x

array([1, 1, 1, 2, 2, 2])

In [4]:
x = np.array([1,2])
x = np.repeat(x, 3)
x

array([1, 1, 1, 2, 2, 2])

### Fuzzification Layer

In [5]:
multiplicator = np.arange(3)
multiplicator = np.tile(np.arange(3), 2)
multiplicator

array([0, 1, 2, 0, 1, 2])

In [6]:
def center_init(x, n_mfs):
    """Initializes the centers of MFs by partitioning the domain of the features

    Args:
        x (numpy.ndarray): the max values for each feature, i.e. the domain
        n_mfs (int): number of MFs in Fuzzification Layer
        
    Returns: 
        numpy.ndarray: initalized widths with the shape (x.size,)
    """
    n_inputs = x.size // n_mfs
    multiplicator = np.tile(np.arange(1, n_mfs + 1), n_inputs)
    return (x / (n_mfs + 1)) * multiplicator

x = np.array([8,10])
x = np.repeat(x, 3)
print(type(x))
print(center_init(x, 3))
print(type(center_init(x, 3))) # numpy.ndarray'

<class 'numpy.ndarray'>
[2.  4.  6.  2.5 5.  7.5]
<class 'numpy.ndarray'>


In [7]:
def widths_init(x, n_mfs):
    """Initializes the widths of MFs by partitioning the domain of the features

    Args:
        x (numpy.ndarray): the max values for each feature, i.e. the feature's domain
        n_mfs (int): number of MFs in Fuzzification Layer

    Returns: 
        numpy.ndarray: initalized widths with the shape (x.size,)
    """
    return x/(n_mfs+1)

x = np.array([8,10])
x = np.repeat(x, 3)
print(x.size)
print(widths_init(x, 3))
print(widths_init(x, 3).shape)

6
[2.  2.  2.  2.5 2.5 2.5]
(6,)


In [8]:
def MF(x, center, width):
    return np.exp(-0.5*(((x-center)/width)**2))
    

In [9]:
x = np.array([1,2])
x = np.repeat(x, 3)

center = np.array([1, 2, 3, 4, 5, 6])# np.ones(3)
width = np.array([1, 2, 3, 4, 5, 6])

print(x)

MF(x, center, width)

[1 1 1 2 2 2]


array([1.        , 0.8824969 , 0.8007374 , 0.8824969 , 0.83527021,
       0.8007374 ])

### Rule Antecedent Layer


In [10]:
#x = np.array([1,2,3,4,5,6])
x = np.array([1,2,3,4,5,6,7,8,9,10,11,12])


x = np.array_split(x, range(3, len(x), 3))



x = np.meshgrid(*x) # the '*' unpacks x and passes to messgrid

x = np.prod(x, axis=0).ravel()

x

array([ 280,  308,  336,  320,  352,  384,  360,  396,  432,  560,  616,
        672,  640,  704,  768,  720,  792,  864,  840,  924, 1008,  960,
       1056, 1152, 1080, 1188, 1296,  350,  385,  420,  400,  440,  480,
        450,  495,  540,  700,  770,  840,  800,  880,  960,  900,  990,
       1080, 1050, 1155, 1260, 1200, 1320, 1440, 1350, 1485, 1620,  420,
        462,  504,  480,  528,  576,  540,  594,  648,  840,  924, 1008,
        960, 1056, 1152, 1080, 1188, 1296, 1260, 1386, 1512, 1440, 1584,
       1728, 1620, 1782, 1944])

In [11]:
feature_names = np.array(["x1", "x2"])
mfs = np.array(["low", "medium", "high"])


x = feature_names
x = np.tile(x[:, np.newaxis], 3).ravel()
mfs = np.tile(mfs, 2)

x = np.char.add(x, ",")    
x = np.char.add(x, mfs)    
x = np.array_split(x, range(3, len(x), 3))
x.reverse()
print(x)
x = np.array(np.meshgrid(*x)) # the '*' unpacks x and passes to messgrid
print(x)
m = x[1].ravel()
m = np.char.add(m, ",")   
x = np.char.add(m, x[0].ravel())   
x = x.tolist()
x

[array(['x2,low', 'x2,medium', 'x2,high'], dtype='<U9'), array(['x1,low', 'x1,medium', 'x1,high'], dtype='<U9')]
[[['x2,low' 'x2,medium' 'x2,high']
  ['x2,low' 'x2,medium' 'x2,high']
  ['x2,low' 'x2,medium' 'x2,high']]

 [['x1,low' 'x1,low' 'x1,low']
  ['x1,medium' 'x1,medium' 'x1,medium']
  ['x1,high' 'x1,high' 'x1,high']]]


['x1,low,x2,low',
 'x1,low,x2,medium',
 'x1,low,x2,high',
 'x1,medium,x2,low',
 'x1,medium,x2,medium',
 'x1,medium,x2,high',
 'x1,high,x2,low',
 'x1,high,x2,medium',
 'x1,high,x2,high']

In [12]:
x = np.array([1,2,3,4,5,6,7,8,9,10,11,12])

x = np.array_split(x, range(3, len(x), 3))
print(x)

x =  np.meshgrid(x[0], x[1], x[2])
#print(x)
x = (x[0] * x[1]).ravel()
x

[array([1, 2, 3]), array([4, 5, 6]), array([7, 8, 9]), array([10, 11, 12])]


array([ 4,  4,  4,  8,  8,  8, 12, 12, 12,  5,  5,  5, 10, 10, 10, 15, 15,
       15,  6,  6,  6, 12, 12, 12, 18, 18, 18])

### rule consequent layer

In [13]:
x = np.array([1,2,3,4,5,6,7,8,9], dtype=np.float64)
weights = np.zeros((x.shape[0], 2), dtype=np.float64)
print(x)
print(weights)

x = x[:, np.newaxis]
print(x)

x= x* weights
x

[1. 2. 3. 4. 5. 6. 7. 8. 9.]
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
[[1.]
 [2.]
 [3.]
 [4.]
 [5.]
 [6.]
 [7.]
 [8.]
 [9.]]


array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

## Training

### error function

In [14]:
output = np.array([[0, 1], [1, 0], [0, 1], [1, 0]], dtype=np.float64)
target = np.array([[1,0], [1, 0], [0, 1], [0, 1]], dtype=np.float64)
output.shape[0]
result = np.zeros(shape=(output.shape[0],), dtype=np.float64)
print(result)

[0. 0. 0. 0.]


In [15]:
# opt 1
def calculate_difference(output, target):
    # Find indices where the value is 1 in the output array
    indices = np.where(output == 1)

    # Create a zero-filled array of the same shape as output
    result = np.zeros(shape=(output.shape[0],), dtype=np.float64)

    # Set the values at the found indices to the difference of output and target
    result = output[indices] - target[indices]

    return result

calculate_difference(output, target)

array([1., 0., 0., 1.])

In [16]:
# opt 2
result = np.zeros_like(output, dtype=np.float64)

# Directly subtract target from output at the found indices
0.5*np.subtract(output, target, where=output==1, out=result)**2

array([[0. , 0.5],
       [0. , 0. ],
       [0. , 0. ],
       [0.5, 0. ]])

### adapt

In [17]:
error = np.ones(shape=(9,))

x1 = np.array([1,2,3])
x2 = np.array([4,5,6])

x =  np.meshgrid(x1, x2)
x # get those directly from antecedent layer with inputs attribute



[array([[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]]),
 array([[4, 4, 4],
        [5, 5, 5],
        [6, 6, 6]])]

In [18]:
# reshape error to match each mu 
error = np.reshape(error, x[0].shape)
print(error)

delta = [mu * error for mu in x]
delta.reverse() # the other mu
print(delta)

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
[array([[4., 4., 4.],
       [5., 5., 5.],
       [6., 6., 6.]]), array([[1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.]])]


In [19]:
# these would result from derived MF function
centers_prime = np.arange(stop=6) 
widths_prime = np.arange(stop=6)

# imitate the whole meshgrid process like in antecedent layer
x = np.array_split(centers_prime, range(3, len(centers_prime), 3))
centers_grided = np.meshgrid(x[0], x[1]) 

x = np.array_split(widths_prime, range(3, len(widths_prime), 3))
widths_grided = np.meshgrid(x[0], x[1]) 

In [20]:
print(delta)
print("\n")
print(centers_grided)
print("\n")
print(widths_grided)

[array([[4., 4., 4.],
       [5., 5., 5.],
       [6., 6., 6.]]), array([[1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.]])]


[array([[0, 1, 2],
       [0, 1, 2],
       [0, 1, 2]]), array([[3, 3, 3],
       [4, 4, 4],
       [5, 5, 5]])]


[array([[0, 1, 2],
       [0, 1, 2],
       [0, 1, 2]]), array([[3, 3, 3],
       [4, 4, 4],
       [5, 5, 5]])]


In [21]:
delta = [d * centers for d,centers in zip(delta,centers_grided)]
delta

[array([[ 0.,  4.,  8.],
        [ 0.,  5., 10.],
        [ 0.,  6., 12.]]),
 array([[ 3.,  6.,  9.],
        [ 4.,  8., 12.],
        [ 5., 10., 15.]])]

In [22]:
centers_to_tune = np.arange(stop=6) 
widths_to_tune = np.arange(stop=6)
print(centers_to_tune)

[0 1 2 3 4 5]


In [23]:
centers_to_tune = np.array_split(centers_to_tune, range(3, len(centers_to_tune), 3))
widths_to_tune = np.array_split(widths_to_tune, range(3, len(widths_to_tune), 3))
centers_to_tune

[array([0, 1, 2]), array([3, 4, 5])]

In [24]:
centers_to_tune[0] = centers_to_tune[0]

In [25]:
error = np.arange(stop=9)

x = np.array([1,2,3,4,5,6,7,8,9])

x = np.array_split(x, range(3, len(x), 3))
print(x)

x =  np.meshgrid(*x, indexing="ij")
x = np.prod(x,axis=0)
x # get those directly from antecedent layer with inputs attribute


[array([1, 2, 3]), array([4, 5, 6]), array([7, 8, 9])]


array([[[ 28,  32,  36],
        [ 35,  40,  45],
        [ 42,  48,  54]],

       [[ 56,  64,  72],
        [ 70,  80,  90],
        [ 84,  96, 108]],

       [[ 84,  96, 108],
        [105, 120, 135],
        [126, 144, 162]]])

In [26]:
# reshape error to match each mu 
error = np.reshape(error, x[0].shape)
print(error)



[[0 1 2]
 [3 4 5]
 [6 7 8]]


In [27]:
delta = [mu * error for mu in x]
delta.reverse() # the other mu
print(delta)

[array([[   0,   96,  216],
       [ 315,  480,  675],
       [ 756, 1008, 1296]]), array([[  0,  64, 144],
       [210, 320, 450],
       [504, 672, 864]]), array([[  0,  32,  72],
       [105, 160, 225],
       [252, 336, 432]])]


In [28]:
deltas = []
for i, _ in enumerate(x):
    print("i", i)
    p = None
    print("p", p)
    p = error
    
    print("e", error)
    for j, mu in enumerate(x):
        
        print("j", j)
        if i==j:
            
            continue
        else:
            p *= mu
            print("Delta", p)
    deltas.append(p)
deltas

i 0
p None
e [[0 1 2]
 [3 4 5]
 [6 7 8]]
j 0
j 1
Delta [[  0  64 144]
 [210 320 450]
 [504 672 864]]
j 2
Delta [[     0   6144  15552]
 [ 22050  38400  60750]
 [ 63504  96768 139968]]
i 1
p None
e [[     0   6144  15552]
 [ 22050  38400  60750]
 [ 63504  96768 139968]]
j 0
Delta [[      0  196608  559872]
 [ 771750 1536000 2733750]
 [2667168 4644864 7558272]]
j 1
j 2
Delta [[         0   18874368   60466176]
 [  81033750  184320000  369056250]
 [ 336063168  668860416 1224440064]]
i 2
p None
e [[         0   18874368   60466176]
 [  81033750  184320000  369056250]
 [ 336063168  668860416 1224440064]]
j 0
Delta [[          0   603979776 -2118184960]
 [-1458786046 -1217134592  -572337934]
 [ 1229751168  2040528896  1695254016]]
j 1
Delta [[          0           0  2109505536]
 [  964191884  1413480448    29193492]
 [  219883008 -1677721600 -1596160000]]
j 2


[array([[          0,           0,  2109505536],
        [  964191884,  1413480448,    29193492],
        [  219883008, -1677721600, -1596160000]]),
 array([[          0,           0,  2109505536],
        [  964191884,  1413480448,    29193492],
        [  219883008, -1677721600, -1596160000]]),
 array([[          0,           0,  2109505536],
        [  964191884,  1413480448,    29193492],
        [  219883008, -1677721600, -1596160000]])]

## decompose

In [29]:
error = np.arange(stop=9)

x = np.array([1,2,3,4,5,6,7,8,9,10,11,12])

x = np.array_split(x, range(3, len(x), 3))


delta =  np.meshgrid(*x, indexing="ij")
print(delta)


[array([[[[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]],

        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]],

        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]]],


       [[[2, 2, 2],
         [2, 2, 2],
         [2, 2, 2]],

        [[2, 2, 2],
         [2, 2, 2],
         [2, 2, 2]],

        [[2, 2, 2],
         [2, 2, 2],
         [2, 2, 2]]],


       [[[3, 3, 3],
         [3, 3, 3],
         [3, 3, 3]],

        [[3, 3, 3],
         [3, 3, 3],
         [3, 3, 3]],

        [[3, 3, 3],
         [3, 3, 3],
         [3, 3, 3]]]]), array([[[[4, 4, 4],
         [4, 4, 4],
         [4, 4, 4]],

        [[5, 5, 5],
         [5, 5, 5],
         [5, 5, 5]],

        [[6, 6, 6],
         [6, 6, 6],
         [6, 6, 6]]],


       [[[4, 4, 4],
         [4, 4, 4],
         [4, 4, 4]],

        [[5, 5, 5],
         [5, 5, 5],
         [5, 5, 5]],

        [[6, 6, 6],
         [6, 6, 6],
         [6, 6, 6]]],


       [[[4, 4, 4],
         [4, 4, 4],
         [4, 4, 4]]

In [30]:
deltas = []
x = delta[0]
x = np.sum(x, axis=3)
x = np.sum(x, axis=1)
x = np.sum(x, axis=1)
deltas.append(x)

x = delta[1]
x = np.sum(x, axis=0)
x = np.sum(x, axis=1)
x = np.sum(x, axis=1)
deltas.append(x)

x = delta[2]
x = np.sum(x, axis=0) # or 1
x = np.sum(x, axis=2)
x = np.sum(x, axis=0)
deltas.append(x)

x = delta[3]
x = np.sum(x, axis=1) # or 2
x = np.sum(x, axis=0)
x = np.sum(x, axis=0)
deltas.append(x)
print(deltas)

[array([27, 54, 81]), array([108, 135, 162]), array([189, 216, 243]), array([270, 297, 324])]


In [31]:
deltas = np.array(deltas)
deltas = deltas.ravel()
deltas

array([ 27,  54,  81, 108, 135, 162, 189, 216, 243, 270, 297, 324])

In [32]:
error = np.arange(stop=9)

x = np.array([1,2,3,4,5,6,7,8,9])

x = np.array_split(x, range(3, len(x), 3))


delta =  np.meshgrid(*x, indexing="ij")
print(delta)


[array([[[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]],

       [[2, 2, 2],
        [2, 2, 2],
        [2, 2, 2]],

       [[3, 3, 3],
        [3, 3, 3],
        [3, 3, 3]]]), array([[[4, 4, 4],
        [5, 5, 5],
        [6, 6, 6]],

       [[4, 4, 4],
        [5, 5, 5],
        [6, 6, 6]],

       [[4, 4, 4],
        [5, 5, 5],
        [6, 6, 6]]]), array([[[7, 8, 9],
        [7, 8, 9],
        [7, 8, 9]],

       [[7, 8, 9],
        [7, 8, 9],
        [7, 8, 9]],

       [[7, 8, 9],
        [7, 8, 9],
        [7, 8, 9]]])]


In [33]:
x = delta[0]
x = np.sum(x, axis=(1,2))


#x = np.sum(x, axis=1)
x

array([ 9, 18, 27])

In [34]:
x = delta[1]
x = np.sum(x, axis=0)
x = np.sum(x, axis=1)
x

array([36, 45, 54])

In [35]:
x = delta[2]
x = np.sum(x, axis=1)
x = np.sum(x, axis=0)
x

array([63, 72, 81])

In [36]:
x = np.array([1,2,3,4,5,6])
x.setflags(write=1) # not needed 
x = np.array_split(x, range(3, len(x), 3))

xi = x
ndim = len(xi)



s0 = (1,) * ndim
output = [np.asanyarray(x).reshape(s0[:i] + (-1,) + s0[i + 1:])
            for i, x in enumerate(xi)]

if ndim > 1:
    # switch first and second axis
    output[0].shape = (1, -1) + s0[2:]
    output[1].shape = (-1, 1) + s0[2:]


# Return the full N-D matrix (not only the 1-D vector)
print("out", output)
output = np.broadcast_arrays(*output, subok=True)

output = output[0]
output.setflags(write=1) # needed 
output += 3
#output = output * 3
print(output)
print(x)

out [array([[1, 2, 3]]), array([[4],
       [5],
       [6]])]
[[4 5 6]
 [4 5 6]
 [4 5 6]]
[array([4, 5, 6]), array([4, 5, 6])]


In [37]:
# params to tune
param = np.array([1,2,3,4,5,6])

param_split = np.array_split(param, range(3, len(param), 3)) # does return adress 

#param_gridded = [p*5 for p in param_gridded] # does not change param
param_split = param_split[0]
param_split *= 5 # changes param


print(param)
print(param_split)

[ 5 10 15  4  5  6]
[ 5 10 15]


In [38]:
# params to tune
param = np.array([1,2,3,4,5,6])

param_split = np.array_split(param, range(3, len(param), 3)) # does return adress 

param_gridded = np.meshgrid(*param_split, indexing="ij", copy=False) # If False, a view into the original arrays are returned in order to conserve memory.  Default is True.
param_gridded = param_gridded[0]
param_gridded.setflags(write=1) # needed 
param_gridded *= 5 # changes param


print(param)
print(param_split)
print(param_gridded)

[ 5 10 15  4  5  6]
[array([ 5, 10, 15]), array([4, 5, 6])]
[[ 5  5  5]
 [10 10 10]
 [15 15 15]]


In [39]:
param = np.array([1,2,3,4,5,6])

param_split = np.array_split(param, range(3, len(param), 3)) # does return adress 

param_gridded = np.meshgrid(*param_split, indexing="ij", copy=False) # If False, a view into the original arrays are returned in order to conserve memory.  Default is True.


print(param)
print(param_split)
print(param_gridded)

[1 2 3 4 5 6]
[array([1, 2, 3]), array([4, 5, 6])]
[array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]]), array([[4, 5, 6],
       [4, 5, 6],
       [4, 5, 6]])]


In [40]:
# params to tune
param = np.array([1,2,3,4,5,6])

param_split = np.array_split(param, range(3, len(param), 3)) # does return adress 

param_gridded = np.meshgrid(*param_split, indexing="ij", copy=False) # If False, a view into the original arrays are returned in order to conserve memory.  Default is True.

for p in param_gridded:
   # param_gridded = param_gridded[0]
    p.setflags(write=1) # needed 
    p *= 5 # changes param
 

print(param)
print(param_split)
print(param_gridded)

[ 5 10 15 20 25 30]
[array([ 5, 10, 15]), array([20, 25, 30])]
[array([[ 5,  5,  5],
       [10, 10, 10],
       [15, 15, 15]]), array([[20, 25, 30],
       [20, 25, 30],
       [20, 25, 30]])]


In [41]:


def _broadcast_to(array, shape, subok, readonly):
    print("arr", array)
    print("shape", shape)
    shape = tuple(shape) if np.iterable(shape) else (shape,)
    array = np.array(array, copy=False, subok=subok)
    if not shape and array.shape:
        raise ValueError('cannot broadcast a non-scalar to a scalar array')
    if any(size < 0 for size in shape):
        raise ValueError('all elements of broadcast shape must be non-'
                         'negative')
    extras = []
    it = np.nditer(
        (array,), flags=['multi_index', 'refs_ok', 'zerosize_ok'] + extras,
        op_flags=['readonly'], itershape=shape, order='C')
    print("it views", it.itviews)

    with it:
        # never really has writebackifcopy semantics
        broadcast = it.itviews[0]
        print("it views", it.itviews)
        print("broad", broadcast)
   # result = _maybe_view_as_subclass(array, broadcast)
    # In a future version this will go away
   # if not readonly and array.flags._writeable_no_warn:
        


    
    broadcast.flags.writeable = True
    broadcast = broadcast.reshape(-1, 1)
   # broadcast.flags._warn_on_write = True
    return broadcast


def _maybe_view_as_subclass(original_array, new_array):
    if type(original_array) is not type(new_array):
        # if input was an ndarray subclass and subclasses were OK,
        # then view the result as that subclass.
        new_array = new_array.view(type=type(original_array))
        # Since we have done something akin to a view from original_array, we
        # should let the subclass finalize (if it has it implemented, i.e., is
        # not None).
        if new_array.__array_finalize__:
            new_array.__array_finalize__(original_array)
    return new_array


def broadcast_to(array, shape, subok=False):
    """Broadcast an array to a new shape.

    Parameters
    ----------
    array : array_like
        The array to broadcast.
    shape : tuple or int
        The shape of the desired array. A single integer ``i`` is interpreted
        as ``(i,)``.
    subok : bool, optional
        If True, then sub-classes will be passed-through, otherwise
        the returned array will be forced to be a base-class array (default).

    Returns
    -------
    broadcast : array
        A readonly view on the original array with the given shape. It is
        typically not contiguous. Furthermore, more than one element of a
        broadcasted array may refer to a single memory location.

    Raises
    ------
    ValueError
        If the array is not compatible with the new shape according to NumPy's
        broadcasting rules.

    See Also
    --------
    broadcast
    broadcast_arrays
    broadcast_shapes

    Notes
    -----
    .. versionadded:: 1.10.0

    Examples
    --------
    >>> x = np.array([1, 2, 3])
    >>> np.broadcast_to(x, (3, 3))
    array([[1, 2, 3],
           [1, 2, 3],
           [1, 2, 3]])
    """
    return _broadcast_to(array, shape, subok=subok, readonly=False)

def _broadcast_shape(*args):
    """Returns the shape of the arrays that would result from broadcasting the
    supplied arrays against each other.
    """
    # use the old-iterator because np.nditer does not handle size 0 arrays
    # consistently
    b = np.broadcast(*args[:32])
    # unfortunately, it cannot handle 32 or more arguments directly
    for pos in range(32, len(args), 31):
        # ironically, np.broadcast does not properly handle np.broadcast
        # objects (it treats them as scalars)
        # use broadcasting to avoid allocating the full array
        b = broadcast_to(0, b.shape)
        b = np.broadcast(b, *args[pos:(pos + 31)])
    return b.shape



def broadcast_arrays(*args, subok=True):
    args = [np.array(_m, copy=False, subok=subok) for _m in args]
    temp = args
    #temp *= 5 
    print(args)
    shape = _broadcast_shape(*args)

    if all(array.shape == shape for array in args):
        # Common case where nothing needs to be broadcasted.
        return args

    return [_broadcast_to(array, shape, subok=subok, readonly=False) # changed
            for array in args]


In [42]:





def meshgrid_custom(*xi,  indexing='ij',copy=False):
    """
    https://github.com/numpy/numpy/blob/v1.26.0/numpy/lib/function_base.py#L5010-L5165
    """
    ndim = len(xi)


    s0 = (1,) * ndim
    output = [np.asanyarray(x).reshape(s0[:i] + (-1,) + s0[i + 1:])
              for i, x in enumerate(xi)]
   # output1 = [np.asanyarray(x).reshape(s0[:i] + (-1,) + s0[i + 1:])
   #           for i, x in enumerate(xi)]
   # output2 = [np.asanyarray(x).reshape(s0[:i] + (-1,) + s0[i + 1:])
   #           for i, x in enumerate(xi)]

    
    # Return the full N-D matrix (not only the 1-D vector)
  #  output = broadcast_arrays(*output, subok=True)
    
    args = [np.array(_m, copy=False, subok=True) for _m in output]
    shape = _broadcast_shape(*args)

   # out = [_broadcast_to(array, shape, subok=True, readonly=False) # changed
     #       for array in args]
    
    out = []
    for array in args:
        
        shape = tuple(shape) if np.iterable(shape) else (shape,)
        array = np.array(array, copy=False, subok=True)
        extras = []
        it = np.nditer(
            (array,), flags=['multi_index', 'refs_ok', 'zerosize_ok'] + extras,
            op_flags=['readonly'], itershape=shape, order='C')
        print("it views", it.itviews)

        with it:
            # never really has writebackifcopy semantics
            broadcast = it.itviews[0]
            print("it views", it.itviews)
            print("broad", broadcast)
    
        broadcast.flags.writeable = True
       # broadcast = broadcast.reshape(-1, 1)
   # broadcast.flags._warn_on_write = True
        out.append(broadcast)




          
    print("Output", out)
    
    
  #  a1  = np.array(output[0], copy=False)    
   # a2 = np.array(output[0], copy=False)    
 #   output[0]  = np.concatenate((output[0], output1[0], output2[0]), axis=1)


   # a3  = np.array(output[1], copy=False)    
  #  a4 = np.array(output[1], copy=False)    
  #  output[1]  = np.concatenate((output[1], output1[1], output2[1]), axis=0)
    return out






In [43]:



def meshgrid_sum_reverse(*xi,  indexing='ij',copy=False):
    """
    https://github.com/numpy/numpy/blob/v1.26.0/numpy/lib/function_base.py#L5010-L5165
    """

    
    args = [np.array(_m, copy=False, subok=True) for _m in xi]
    print("args", args)
    shape = _broadcast_shape(*args)
    out = []
    for array in args:
        
        shape = tuple(shape) if np.iterable(shape) else (shape,)
        array = np.array(array, copy=False, subok=True)
        extras = []
        it = np.nditer(
            (array,), flags=['multi_index', 'refs_ok', 'zerosize_ok'] + extras,
            op_flags=['readonly'], itershape=shape, order='C')
        print("it views", it.itviews)

        with it:
            # never really has writebackifcopy semantics
            broadcast = it.itviews[0]
            print("it views", it.itviews)
            print("broad", broadcast)
    
        broadcast.flags.writeable = True
       # broadcast = broadcast.reshape(-1, 1)
   # broadcast.flags._warn_on_write = True
        out.append(broadcast)

    ndim = len(xi)
    s0 = (1,) * ndim
    print("HUH", s0)
    print("WHAT", s0[:i] + (-1,) + s0[i + 1:])
    output = [np.asanyarray(x).sum(s0[:i] + (-1,) + s0[i + 1:])
              for i, x in enumerate(out)]
    
    print("oUT", output)

    return output

In [44]:

# params to tune
param = np.array([1,2,3,4,5,6])
print(param)

param_split = np.array_split(param, range(3, len(param), 3)) # does return adress 

param_gridded = meshgrid_custom(*param_split, indexing='ij', copy=False) # If False, a view into the original arrays are returned in order to conserve memory.  Default is True.
print("here",param_gridded)


res = meshgrid_sum_reverse(param_gridded)
res

[1 2 3 4 5 6]
it views (array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]]),)
it views (array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]]),)
broad [[1 1 1]
 [2 2 2]
 [3 3 3]]
it views (array([[4, 5, 6],
       [4, 5, 6],
       [4, 5, 6]]),)
it views (array([[4, 5, 6],
       [4, 5, 6],
       [4, 5, 6]]),)
broad [[4 5 6]
 [4 5 6]
 [4 5 6]]
Output [array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]]), array([[4, 5, 6],
       [4, 5, 6],
       [4, 5, 6]])]
here [array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]]), array([[4, 5, 6],
       [4, 5, 6],
       [4, 5, 6]])]
args [array([[[1, 1, 1],
        [2, 2, 2],
        [3, 3, 3]],

       [[4, 5, 6],
        [4, 5, 6],
        [4, 5, 6]]])]
it views (array([[[1, 1, 1],
        [2, 2, 2],
        [3, 3, 3]],

       [[4, 5, 6],
        [4, 5, 6],
        [4, 5, 6]]]),)
it views (array([[[1, 1, 1],
        [2, 2, 2],
        [3, 3, 3]],

       [[4, 5, 6],
        [4, 5, 6],
        [4, 5, 6]]]),)
broad [[[1 1 1]
  [2 2 2]
 

[array([[ 3,  6,  9],
        [15, 15, 15]])]

In [45]:
# params to tune
param = np.array([1,2,3,4,5,6])
print(param)

param_split = np.array_split(param, range(3, len(param), 3)) # does return adress 

param_gridded = meshgrid_custom(*param_split, indexing='ij', copy=False) # If False, a view into the original arrays are returned in order to conserve memory.  Default is True.
print("here",param_gridded)

delta = [np.array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]]), np.array([[ 10, 11, 12],
       [ 13, 14, 15],
       [ 16, 17, 18]])]

#delta = [np.array(d).reshape(-1, 1) for d in delta]

for p, d in zip(param_gridded,delta):
   # param_gridded = param_gridded[0]
   # for _p, _d in zip(p,d):
              #temp = []
             # for __p, __d in zip(_p,_d):
       p.setflags(write=1) # needed 
       print("_p", p)
       print("__d",d)
       p += d # changes param
       print("p beyond", p)
                     
 

print("after.........")
print(param)
print(param_split)
print(param_gridded)

[1 2 3 4 5 6]
it views (array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]]),)
it views (array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]]),)
broad [[1 1 1]
 [2 2 2]
 [3 3 3]]
it views (array([[4, 5, 6],
       [4, 5, 6],
       [4, 5, 6]]),)
it views (array([[4, 5, 6],
       [4, 5, 6],
       [4, 5, 6]]),)
broad [[4 5 6]
 [4 5 6]
 [4 5 6]]
Output [array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]]), array([[4, 5, 6],
       [4, 5, 6],
       [4, 5, 6]])]
here [array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]]), array([[4, 5, 6],
       [4, 5, 6],
       [4, 5, 6]])]
_p [[1 1 1]
 [2 2 2]
 [3 3 3]]
__d [[1 2 3]
 [4 5 6]
 [7 8 9]]
p beyond [[ 4  4  4]
 [ 8  8  8]
 [12 12 12]]
_p [[4 5 6]
 [4 5 6]
 [4 5 6]]
__d [[10 11 12]
 [13 14 15]
 [16 17 18]]
p beyond [[20 22 24]
 [20 22 24]
 [20 22 24]]
after.........
[ 4  8 12 20 22 24]
[array([ 4,  8, 12]), array([20, 22, 24])]
[array([[ 4,  4,  4],
       [ 8,  8,  8],
       [12, 12, 12]]), array([[20, 22, 24],
       [20, 22, 24],

In [60]:

x = np.array([1,2,3,4,5,6,7,8,9])

mid = np.array_split(x, range(3, len(x), 3)) # does not return adress 
mid[0] = mid[0].reshape(-1, 1)
hm = mid[0] * mid[1] * mid[2].reshape(-1, 1)
hm

array([[ 28,  35,  42],
       [ 64,  80,  96],
       [108, 135, 162]])

In [62]:
x = np.array([1,2,3,4,5,6])
n = 4
best_indeces = np.argsort(x)[-n:] 
best_indeces

array([2, 3, 4, 5], dtype=int64)

In [63]:
x = np.array([1,2,3,4,5,6])

mid = np.array_split(x, range(3, len(x), 3)) # does not return adress 
mid =  np.meshgrid(*mid, indexing="ij")

# out = mid[0]
# out.setflags(write=1) # needed 
# out += 3
print(mid)
mid = np.prod(mid,axis=0)

print(mid)
print(x)


[array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]]), array([[4, 5, 6],
       [4, 5, 6],
       [4, 5, 6]])]
[[ 4  5  6]
 [ 8 10 12]
 [12 15 18]]
[1 2 3 4 5 6]


In [64]:
x = np.array([1,2,3,4,5,6])

mid = np.array_split(x, range(3, len(x), 3)) # does not return adress 
mid =  np.meshgrid(*mid, indexing="ij")

positions = np.vstack(map(np.ravel, mid))
positions

TypeError: arrays to stack must be passed as a "sequence" type such as list or tuple.

In [ ]:
one = np.array([[0.71659513, 0.        ],
       [0.        , 0.47190284],
       [0.        , 0.47169245],
       [0.71323643, 0.        ],
       [0.        , 0.46969102],
       [0.        , 0.46948161],
       [0.97967696, 0.        ],
       [0.64515139, 0.        ],
       [0.64486374, 0.        ]])

two = np.array([[0.75135812, 0.        ],
       [0.        , 0.65214701],
       [0.        , 0.65205271],
       [0.74836295, 0.        ],
       [0.        , 0.64954733],
       [0.        , 0.6494534 ],
       [0.88774161, 0.        ],
       [0.7705221 , 0.        ],
       [0.77041068, 0.        ]])



arr = [one,two]
x = np.concatenate(arr, axis=1)
print(x)

res = np.sum(x, axis =1)
res

[[0.71659513 0.         0.75135812 0.        ]
 [0.         0.47190284 0.         0.65214701]
 [0.         0.47169245 0.         0.65205271]
 [0.71323643 0.         0.74836295 0.        ]
 [0.         0.46969102 0.         0.64954733]
 [0.         0.46948161 0.         0.6494534 ]
 [0.97967696 0.         0.88774161 0.        ]
 [0.64515139 0.         0.7705221  0.        ]
 [0.64486374 0.         0.77041068 0.        ]]


array([1.46795325, 1.12404985, 1.12374516, 1.46159938, 1.11923835,
       1.11893501, 1.86741857, 1.41567349, 1.41527442])